In [ ]:
from captcha.image import ImageCaptcha
import matplotlib.pyplot as plt
from IPython.display import Image
import numpy as np
import random
import string

from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Dense, Flatten, Dropout 
from keras.models import Model, Sequential, load_model
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.utils.vis_utils import plot_model

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

dig_asc = string.digits + string.ascii_uppercase
print(dig_asc)

width, height, n_len, n_class = 170, 80, 4, len(dig_asc)

cap_gen = ImageCaptcha(width=width, height=height)
cap_str = ''.join([random.choice(dig_asc) for i in range(4)])
cap_img = cap_gen.generate_image(cap_str)

plt.imshow(cap_img)
plt.title('Real: %s' %(cap_str))

In [ ]:
def genCap(batch_size=32):
  X = np.zeros((batch_size, height, width, 3), dtype=np.uint8)
  y = [np.zeros((batch_size, n_class), dtype=np.uint8) for i in range(n_len)]

  cap_gen = ImageCaptcha(width=width, height=height)
  while True:
    for i in range(batch_size):
      cap_str = ''.join([random.choice(dig_asc) for i in range(n_len)])
      X[i] = cap_gen.generate_image(cap_str)
      for pos, char in enumerate(cap_str):
        y[pos][i, :] = 0
        y[pos][i, dig_asc.find(char)] = 1
    yield X, y

def decode(y):
  y = np.argmax(np.array(y), axis=2)[:,0]
  return ''.join([dig_asc[x] for x in y])

X, y = next(genCap(1))
plt.imshow(X[0])
plt.title(decode(y))

In [ ]:
from keras.models import *
from keras.layers import *

input_tensor = Input(shape=(height, width, 3))
x = input_tensor
for i in range(4):
    x = Conv2D(filters=32*2**i, kernel_size=(3, 3), activation='relu')(x)
    x = Conv2D(filters=32*2**i, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization(axis=3)(x)
    x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = [Dense(n_class, activation='softmax', name='D%d'%(n+1))(x) for n in range(4)]
model = Model(inputs=input_tensor, outputs=x)

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

plot_model(model, to_file='model_captcha.png', show_shapes=True, show_layer_names=True)
Image('model_captcha.png')

In [ ]:
model.fit_generator(genCap(), steps_per_epoch=10000, epochs=5,
                    validation_data=genCap(), validation_steps=1000)

In [ ]:
X, y = next(genCap(1))
y_pred = model.predict(X)
plt.title('real: %s\n pred: %s'%(decode(y), decode(y_pred)))
plt.imshow(X[0])